In [ ]:
import os
import pandas as pd

# Grade Processor

TODO:
1. Replace duplicated IDs with thier actual IDs?

### Unzip files

In [ ]:
RAW_GRADES_PATH = 'data/grades/raw'

for root, _, files in os.walk(RAW_GRADES_PATH):
    for file in files:
        if not file.endswith('.zip'):
            continue
    
        path = os.path.join(root, file)
        try:
            os.system('unzip ' + path + " -d ./ >/dev/null 2>&1")
            os.remove(path)
        except FileNotFoundError:
            print("Failed to unzip: ", path)
            continue

In [ ]:
def process_csv(path, skill, participant_id, batch):
    temp_df = pd.read_csv(path,
                          usecols=['Name', 'A: A++', 'A: A+', 'A: A', 'A: A-', 
                                   'B: B+', 'B: B', 'B: B-', 
                                   'C: C+', 'C: C', 'C: C-',
                                   'D: D+', 'D: D', 'D: D-',
                                   'F', 'Comments'])
    temp_df['skill'] = skill
    temp_df['participant_id'] = participant_id
    temp_df['batch'] = batch
    temp_df = temp_df.iloc[:20]
    
    for col in temp_df.columns:
        if ':' in col :
            temp_df[col] = temp_df[col].astype(str).str.upper().replace('TRUE', col.split(': ')[1])
        elif col == 'F':
            temp_df[col] = temp_df[col].astype(str).str.upper().replace('TRUE', 'F')
        else:
            continue
            
        temp_df[col] = temp_df[col].astype(str).str.upper().replace('FALSE', None)
    
    grade_col = (temp_df['A: A++'].combine_first(temp_df['A: A+'])
                 .combine_first(temp_df['A: A'])
                 .combine_first(temp_df['A: A-'])
                 .combine_first(temp_df['B: B+'])
                 .combine_first(temp_df['B: B'])
                 .combine_first(temp_df['B: B-'])
                 .combine_first(temp_df['C: C+'])
                 .combine_first(temp_df['C: C'])
                 .combine_first(temp_df['C: C-'])
                 .combine_first(temp_df['D: D+'])
                 .combine_first(temp_df['D: D'])
                 .combine_first(temp_df['D: D-'])
                 # .combine_first(temp_df['F'])
                 )
    
    temp_df['grade'] = temp_df['F'].combine_first(grade_col)
    temp_df.drop(columns=['A: A++', 'A: A+', 'A: A', 'A: A-',
                          'B: B+', 'B: B', 'B: B-',
                          'C: C+', 'C: C', 'C: C-',
                          'D: D+', 'D: D', 'D: D-',
                          'F'], inplace=True)
    
    return temp_df

In [ ]:
grade_df = None

for root, _, files in os.walk(RAW_GRADES_PATH):
    split = root.split('_')
    
    if len(split) == 1:
        continue
    
    participant_id = split[4]
    
    if len(split) == 9:
        batch = 1
    else:
        batch = 2
        
    for file in files:
        skill = file[2:-4].replace('_', ' ')
        
        if 'Correctness' in file or 'Code_Elegance' in file or 'Readability' in file or 'Documentation' in file:
            try:
                if grade_df is None:
                    grade_df = process_csv(os.path.join(root, file), skill, participant_id, batch)
                else:
                    grade_df = pd.concat([grade_df, process_csv(os.path.join(root, file), skill, participant_id, batch)])
            except ValueError as e:
                print('Error processing: ', os.path.join(root, file))
                print(e)

grade_df.rename(columns={'Name': 'assignment_number', 'Comments': 'comments'}, inplace=True)
grade_df

#### Assignments Missing Grades

In [ ]:
grade_df[grade_df['grade'].isnull()]

In [ ]:
grade_df.to_csv('data/grades.csv')